In [64]:
import io
import itertools
import numpy as np
import sklearn.metrics
from tensorboard.plugins.hparams import api as hp
import matplotlib.pyplot as plt

import tensorflow as tf

In [66]:
data_train=np.load("../Downloads/9. Practical Project/Dataset/Primary categories - Train.npz")
data_test=np.load("../Downloads/9. Practical Project/Dataset/Primary categories - Test.npz")
data_validation=np.load("../Downloads/9. Practical Project/Dataset/Primary categories - Validation.npz")

In [69]:
images_train=data_train['images']
label_train=data_train['labels']

images_test=data_test['images']
label_test=data_test['labels']


In [72]:
images_val=data_validation['images']
label_val=data_validation['labels']

In [74]:
images_train=images_train/255.0
images_val=images_val/255.0
images_test=images_test/255.0

In [75]:
EPOCHS=15
BATCH_SIZE=64

In [94]:
HP_FILTER_SIZE=hp.HParam('filter_size',hp.Discrete([3,5,7]))
HP_FILTER_NUM=hp.HParam('filter_number',hp.Discrete([32,64,96,128]))

METRIC_ACCURACY='accuracy'

with tf.summary.create_file_writer(r'logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE,HP_FILTER_NUM],
        metrics=[hp.Metric(METRIC_ACCURACY,display_name='Accuracy')],
    )

In [ ]:
def train_test_model(hparams,session_num):
    model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM],hparams[HP_FILTER_SIZE],activation='relu',input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM],3,activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3)
    ])
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer='adam',loss=loss_fn,metrics=['accuracy'])
    log_dir="Logs\\Model 1\\fit]\\"+"run-{}".format(sesssession_num)
    

In [ ]:
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

    # Defining the logging directory
    log_dir = "Logs\\MOdel 1\\fit\\" + "run-{}".format(session_num)
    
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')     
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(label_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Glasses/Sunglasses', 'Trousers/Jeans', 'Shoes'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        label_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,label_val),
        verbose = 2
    )
    
    
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(images_val,label_val)
    
    # Saving the current model for future reference
    # Saving the current model for future reference
    model.save(r"saved_models\\Model 1\\Run-{}".format(session_num))
 # Doubling backslashes

    
    return accuracy

In [ ]:
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [115]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:

        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_FILTER_NUM: filter_num
        }

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)

        session_num += 1

--- Starting trial: run-1
{'filter_size': 3, 'filter_number': 32}


/home/bizmotion/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
203/203 - 47s - 231ms/step - accuracy: 0.9806 - loss: 0.0709 - val_accuracy: 0.9994 - val_loss: 0.0042
Epoch 2/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
203/203 - 80s - 392ms/step - accuracy: 0.9982 - loss: 0.0115 - val_accuracy: 1.0000 - val_loss: 0.0029
Epoch 3/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
203/203 - 46s - 228ms/step - accuracy: 0.9987 - loss: 0.0090 - val_accuracy: 0.9969 - val_loss: 0.0122
Epoch 4/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
203/203 - 45s - 223ms/step - accuracy: 0.9996 - loss: 0.0036 - val_accuracy: 0.9981 - val_loss: 0.0068
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 1.0000 - loss: 0.0024


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=saved_models\\Model 1\\Run-1.

In [117]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/hrapam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
